# Escuela Politécnica Nacional
**Nombres:** Mateo Coronado  

**Fecha:** 4 de junio 2025

**Materia:** INTELIGENCIA ARTIFICIAL


## PROYECTO DE INTELIGENCIA ARTIFICIAL

### PRIMER BIMESTRE  
### Proyecto 2: Rompecabezas Lógico Usando Lógica Proposicional

**Descripción del Proyecto**

**OBJETIVOS:**

El objetivo de este proyecto es implementar y resolver un rompecabezas lógico utilizando lógica proposicional. Los estudiantes deberán aplicar técnicas de razonamiento lógico para identificar, entre varios sospechosos, quién es culpable de un crimen. Se proporciona un conjunto de declaraciones, sabiendo que exactamente una de ellas es falsa y las otras dos son verdaderas.

**Rompecabezas**

**Problema 1:**  
Determine quién entre los siguientes es culpable de dopaje. Los sospechosos son: Alice, Bob y Charlie.

**Declaraciones:**

- Alice dijo: “Bob o Charlie tomaron drogas, pero no ambos.”
- Bob dijo: “Alice o Charlie tomaron drogas, pero no ambos.”
- Charlie dijo: “Alice o Bob tomaron drogas, pero no ambos.”

> Nota: De estas tres declaraciones, exactamente dos son verdaderas y una es falsa.

**Instrucciones:**  
Utiliza lógica proposicional para modelar el problema, analiza las declaraciones y determina quién es el culpable. Explica tu razonamiento paso a paso y presenta la solución final.

**Descripción del método:**

**Metodología para la implementación de la lógica proposicional:**

1. Implementar la red bayesiana para el problema de la alarma usando el código proporcionado en Python. 

Resolver:

- Calcular:  
    - P(j, m, ~a, b, ~e)
    - P(~a, b, e)
    - P(b | a)
    - Probabilidad de b dado que llama John, para 10,000 ocurrencias.

Explica cada paso y muestra los resultados obtenidos.

In [2]:
from pomegranate import *

# Nodo: Robo (sin padres)
robo = Node(DiscreteDistribution({
    "sí": 0.001,
    "no": 0.999
}), name="robo")

# Nodo: Terremoto (sin padres)
terremoto = Node(DiscreteDistribution({
    "sí": 0.002,
    "no": 0.998
}), name="terremoto")

# Nodo: Alarma (condicional a Robo y Terremoto)
alarma = Node(ConditionalProbabilityTable([
    ["sí", "sí", "sí", 0.95],
    ["sí", "sí", "no", 0.05],
    ["sí", "no", "sí", 0.94],
    ["sí", "no", "no", 0.06],
    ["no", "sí", "sí", 0.29],
    ["no", "sí", "no", 0.71],
    ["no", "no", "sí", 0.001],
    ["no", "no", "no", 0.999]
], [robo.distribution, terremoto.distribution]), name="alarma")

# Nodo: John llama (condicional a alarma)
john_llama = Node(ConditionalProbabilityTable([
    ["sí", "sí", 0.9],
    ["sí", "no", 0.1],
    ["no", "sí", 0.05],
    ["no", "no", 0.95]
], [alarma.distribution]), name="john_llama")

# Nodo: Mary llama (condicional a alarma)
mary_llama = Node(ConditionalProbabilityTable([
    ["sí", "sí", 0.7],
    ["sí", "no", 0.3],
    ["no", "sí", 0.01],
    ["no", "no", 0.99]
], [alarma.distribution]), name="mary_llama")

# Crear red bayesiana y añadir nodos
modelo = BayesianNetwork("Modelo de la Alarma")
modelo.add_states(robo, terremoto, alarma, john_llama, mary_llama)

# Añadir conexiones (aristas)
modelo.add_edge(robo, alarma)
modelo.add_edge(terremoto, alarma)
modelo.add_edge(alarma, john_llama)
modelo.add_edge(alarma, mary_llama)

# Finalizar red
modelo.bake()


In [7]:
#from model import modelo  # Asegúrate de que el archivo se llama model.py y contiene el modelo

# Evidencia: John ha llamado
predictions = modelo.predict_proba({
    "john_llama": "sí"
})

# Imprimir predicciones para cada nodo
for node, prediction in zip(modelo.states, predictions):
    if isinstance(prediction, str):
        print(f"{node.name}: {prediction}")
    else:
        print(f"{node.name}:")
        for value, prob in prediction.parameters[0].items():
            print(f"    {value}: {prob:.4f}")


robo:
    sí: 0.0163
    no: 0.9837
terremoto:
    sí: 0.0114
    no: 0.9886
alarma:
    sí: 0.0434
    no: 0.9566
john_llama: sí
mary_llama:
    sí: 0.0400
    no: 0.9600


In [12]:
#from model import modelo  # Asegúrate de que está importando bien
import pomegranate
from collections import Counter

def generate_sample():
    sample = {}
    parents = {}
    for state in modelo.states:
        if isinstance(state.distribution, pomegranate.ConditionalProbabilityTable):
            sample[state.name] = state.distribution.sample(parent_values=parents)
        else:
            sample[state.name] = state.distribution.sample()
        parents[state.distribution] = sample[state.name]
    return sample

# Simulaciones
N = 10000
count1 = count2 = count3_num = count3_den = 0
count4_num = count4_den = 0

for _ in range(N):
    s = generate_sample()

    # P(j, m, ~a, b, ~e)
    if s["john_llama"] == "sí" and s["mary_llama"] == "sí" and s["alarma"] == "no" and s["robo"] == "sí" and s["terremoto"] == "no":
        count1 += 1

    # P(~a, b, e)
    if s["alarma"] == "no" and s["robo"] == "sí" and s["terremoto"] == "sí":
        count2 += 1

    # P(b | a)
    if s["alarma"] == "sí":
        count3_den += 1
        if s["robo"] == "sí":
            count3_num += 1

    # P(b | j)
    if s["john_llama"] == "sí":
        count4_den += 1
        if s["robo"] == "sí":
            count4_num += 1

# Mostrar resultados
print(f"P(j,m,~a,b,~e) ≈ {count1 / N:.6f}")
print(f"P(~a,b,e) ≈ {count2 / N:.6f}")
print(f"P(b | a) ≈ {count3_num / count3_den:.6f}" if count3_den > 0 else "No hubo muestras con alarma = sí")
print(f"P(b | j) ≈ {count4_num / count4_den:.6f}" if count4_den > 0 else "No hubo muestras con john_llama = sí")


P(j,m,~a,b,~e) ≈ 0.000000
P(~a,b,e) ≈ 0.000000
P(b | a) ≈ 0.368421
P(b | j) ≈ 0.021739


2. Usando el código proporcionado en el aula virtual para el problema del tren, resolver: 

- P(heavy, yes, delayed, attend)

- P(none, no, on time, miss)

- P(none, yes, delayed)

-P(none|miss)

In [8]:
from pomegranate import *

# Rain node has no parents
rain = Node(DiscreteDistribution({
    "none": 0.7,
    "light": 0.2,
    "heavy": 0.1
}), name="rain")

# Track maintenance node is conditional on rain
maintenance = Node(ConditionalProbabilityTable([
    ["none", "yes", 0.4],
    ["none", "no", 0.6],
    ["light", "yes", 0.2],
    ["light", "no", 0.8],
    ["heavy", "yes", 0.1],
    ["heavy", "no", 0.9]
], [rain.distribution]), name="maintenance")

# Train node is conditional on rain and maintenance
train = Node(ConditionalProbabilityTable([
    ["none", "yes", "on time", 0.8],
    ["none", "yes", "delayed", 0.2],
    ["none", "no", "on time", 0.9],
    ["none", "no", "delayed", 0.1],
    ["light", "yes", "on time", 0.6],
    ["light", "yes", "delayed", 0.4],
    ["light", "no", "on time", 0.7],
    ["light", "no", "delayed", 0.3],
    ["heavy", "yes", "on time", 0.4],
    ["heavy", "yes", "delayed", 0.6],
    ["heavy", "no", "on time", 0.5],
    ["heavy", "no", "delayed", 0.5],
], [rain.distribution, maintenance.distribution]), name="train")

# Appointment node is conditional on train
appointment = Node(ConditionalProbabilityTable([
    ["on time", "attend", 0.9],
    ["on time", "miss", 0.1],
    ["delayed", "attend", 0.6],
    ["delayed", "miss", 0.4]
], [train.distribution]), name="appointment")

# Create a Bayesian Network and add states
model = BayesianNetwork()
model.add_states(rain, maintenance, train, appointment)

# Add edges connecting nodes
model.add_edge(rain, maintenance)
model.add_edge(rain, train)
model.add_edge(maintenance, train)
model.add_edge(train, appointment)

# Finalize model
model.bake()


In [10]:
from pomegranate import *

# Rain node has no parents
rain = Node(DiscreteDistribution({
    "none": 0.7,
    "light": 0.2,
    "heavy": 0.1
}), name="rain")

# Track maintenance node is conditional on rain
maintenance = Node(ConditionalProbabilityTable([
    ["none", "yes", 0.4],
    ["none", "no", 0.6],
    ["light", "yes", 0.2],
    ["light", "no", 0.8],
    ["heavy", "yes", 0.1],
    ["heavy", "no", 0.9]
], [rain.distribution]), name="maintenance")

# Train node is conditional on rain and maintenance
train = Node(ConditionalProbabilityTable([
    ["none", "yes", "on time", 0.8],
    ["none", "yes", "delayed", 0.2],
    ["none", "no", "on time", 0.9],
    ["none", "no", "delayed", 0.1],
    ["light", "yes", "on time", 0.6],
    ["light", "yes", "delayed", 0.4],
    ["light", "no", "on time", 0.7],
    ["light", "no", "delayed", 0.3],
    ["heavy", "yes", "on time", 0.4],
    ["heavy", "yes", "delayed", 0.6],
    ["heavy", "no", "on time", 0.5],
    ["heavy", "no", "delayed", 0.5],
], [rain.distribution, maintenance.distribution]), name="train")

# Appointment node is conditional on train
appointment = Node(ConditionalProbabilityTable([
    ["on time", "attend", 0.9],
    ["on time", "miss", 0.1],
    ["delayed", "attend", 0.6],
    ["delayed", "miss", 0.4]
], [train.distribution]), name="appointment")

# Create a Bayesian Network and add states
model = BayesianNetwork()
model.add_states(rain, maintenance, train, appointment)

# Add edges connecting nodes
model.add_edge(rain, maintenance)
model.add_edge(rain, train)
model.add_edge(maintenance, train)
model.add_edge(train, appointment)

# Finalize model
model.bake()


In [11]:
from collections import Counter
#from model import model
import pomegranate

def generate_sample():
    sample = {}
    parents = {}
    for state in model.states:
        if isinstance(state.distribution, pomegranate.ConditionalProbabilityTable):
            sample[state.name] = state.distribution.sample(parent_values=parents)
        else:
            sample[state.name] = state.distribution.sample()
        parents[state.distribution] = sample[state.name]
    return sample

# Variables para contar
N = 10000
count1 = count2 = count3 = 0
count4_num = count4_den = 0

for _ in range(N):
    s = generate_sample()

    # P(heavy, yes, delayed, attend)
    if s["rain"] == "heavy" and s["maintenance"] == "yes" and s["train"] == "delayed" and s["appointment"] == "attend":
        count1 += 1

    # P(none, no, on time, miss)
    if s["rain"] == "none" and s["maintenance"] == "no" and s["train"] == "on time" and s["appointment"] == "miss":
        count2 += 1

    # P(none, yes, delayed)
    if s["rain"] == "none" and s["maintenance"] == "yes" and s["train"] == "delayed":
        count3 += 1

    # P(none | miss)
    if s["appointment"] == "miss":
        count4_den += 1
        if s["rain"] == "none":
            count4_num += 1

# Mostrar resultados
print(f"P(heavy, yes, delayed, attend) ≈ {count1 / N:.4f}")
print(f"P(none, no, on time, miss) ≈ {count2 / N:.4f}")
print(f"P(none, yes, delayed) ≈ {count3 / N:.4f}")
if count4_den > 0:
    print(f"P(none | miss) ≈ {count4_num / count4_den:.4f}")
else:
    print("No hubo muestras con appointment = miss")


P(heavy, yes, delayed, attend) ≈ 0.0031
P(none, no, on time, miss) ≈ 0.0387
P(none, yes, delayed) ≈ 0.0563
P(none | miss) ≈ 0.6115
